# `microdf` demo

## Setup

In [1]:
import numpy as np
import pandas as pd

import taxcalc as tc
import microdf as mdf

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

## Generate data

In [2]:
base = mdf.calc_df(group_vars=['expanded_income', 'MARS'],
                   metric_vars=['aftertax_income', 'XTOT'])

In [3]:
base.columns

Index(['vet_ben', 's006', 'tanf_ben', 'e02400', 'expanded_income', 'ssi_ben',
       'wic_ben', 'aftertax_income', 'housing_ben', 'e02300', 'mcare_ben',
       'XTOT', 'snap_ben', 'MARS', 'other_ben', 'mcaid_ben', 'market_income',
       'bens', 'tax', 's006_m', 'XTOT_m', 'aftertax_income_m'],
      dtype='object')

Define a reform that treats capital gains as ordinary income and sets the top marginal rate to 70%.

In [4]:
CG_REFORM = {
    'CG_nodiff': {2019: True},
    'II_rt7': {2019: 0.7}
}

In [5]:
reform = mdf.calc_df(reform=CG_REFORM, group_vars=['MARS'], group_n65=True, 
                     metric_vars=['aftertax_income', 'XTOT'])

In [6]:
reform.columns

Index(['s006', 'ssi_ben', 'e02300', 'aftertax_income', 'mcare_ben',
       'expanded_income', 'wic_ben', 'XTOT', 'MARS', 'mcaid_ben', 'vet_ben',
       'tanf_ben', 'e02400', 'housing_ben', 'snap_ben', 'other_ben',
       'market_income', 'bens', 'tax', 'n65', 's006_m', 'XTOT_m',
       'aftertax_income_m'],
      dtype='object')

### Calculate senior UBI.

Start with total revenue ($ billions).

In [7]:
new_rev_m = base.aftertax_income_m.sum() - reform.aftertax_income_m.sum()
new_rev_m / 1e3

337.44427805705

How many seniors are there?

In [8]:
mdf.add_weighted_metrics(reform, 'n65')

n65_total_m = reform.n65_m.sum()
n65_total_m

60.66105169000002

Divide.

In [9]:
senior_ubi = new_rev_m / reform.n65_m.sum()
senior_ubi

5562.7831805738

### Add senior UBI to `aftertax_income` and recalculate

In [10]:
reform['ubi'] = senior_ubi * reform.n65
reform['aftertax_income'] = reform.aftertax_income + reform.ubi
mdf.add_weighted_metrics(reform, 'aftertax_income')

In [11]:
np.allclose(base.aftertax_income_m.sum(), reform.aftertax_income_m.sum())

True

## Analyze

Gini, FPL, distributional impact chart

### Change to Gini index

In [12]:
mdf.gini(base, 'aftertax_income', 's006')

0.4810691765589658

In [13]:
mdf.gini(reform, 'aftertax_income', 's006')

0.464648453442316

### Change to poverty rate

Add federal poverty line with `mdf.fpl`.

In [14]:
base['fpl'] = mdf.fpl(base.XTOT)
reform['fpl'] = mdf.fpl(reform.XTOT)

base['fpl_XTOT_m'] = np.where(base.aftertax_income < base.fpl,
                              base.XTOT_m, 0)
reform['fpl_XTOT_m'] = np.where(reform.aftertax_income < reform.fpl,
                                reform.XTOT_m, 0)

In [15]:
reform.fpl_XTOT_m.sum() / base.fpl_XTOT_m.sum() - 1

-0.04505398741648947